In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import random
import os
import sklearn.cluster 

import msg_passing
import utils
import run
import display
import parse_data
import run

In [ ]:

def predict_links():
    war_fname = "output/gun_regulations_v2_raw_answer_processed_names_uniform_init_update_weights_30K_lr_3_dim_10.graphml" 
    war_fname_hist = "output/gun_regulations_v2_raw_answer_processed_names_uniform_init_update_weights_30K_lr_3_dim_10.pkl" 
    war_fname = "output/combined_v2_general_msg_passing_batch_10_path_10_100K_lr_3_dim_3.graphml" 
    war_fname_hist = "output/combined_v2_general_msg_passing_batch_10_path_10_100K_lr_3_dim_3.pkl" 
    wg = msg_passing.load_graph_graphml(war_fname)
    pg, aux = msg_passing.prune_graph(wg)
    avg_g = msg_passing.avg_edge_weights(pg)
    hist, diagnostic_hist = msg_passing.load_history(war_fname_hist)
    def sign(n):
        if(n > 0):
            return 1
        elif(n < 0):
            return -1
        else:
            return 0

    outliers = []
    for n in pg.nodes():
        for n2 in pg.neighbors(n):
            weights = np.array([v["weight"] for v in wg[n][n2].values()])
            avg_weight = np.average(weights)

            angle = utils.between_angle(pg.nodes()[n]["value"], pg.nodes()[n2]["value"])
            if(abs(avg_weight) > 0.1 or abs(np.pi/2-angle) < 1):
                continue
            if(sign((np.pi/2) - angle) != sign(avg_weight)):
                outliers.append((n, n2, avg_weight, (np.pi/2-angle)/1.5))

    print(len(outliers))
    print(outliers)